## My inital attept at trying to run through a Kaggle competition
Found https://www.kaggle.com/c/web-traffic-time-series-forecasting/kernels?sortBy=votes&after=false


## Setup

In [40]:
from theano.sandbox import cuda

from imp import reload
%matplotlib inline
import utils; reload(utils)
from utils import *
from __future__ import division, print_function
from random import randint

import numpy as np
import h5py
import pandas as pd
import os, os.path
import math

## needed for the date functions
import datetime

time: 11.4 ms


In [41]:
get_ipython().magic('load_ext cellevents')

The cellevents extension is already loaded. To reload it, use:
  %reload_ext cellevents
time: 1.37 ms


In [42]:
# No sample
#path = "/home/jd/data/fish/sample/"
path = "/home/jd/data/webTraffic/"
batch_size=64

time: 2.58 ms


## Load Data

In [43]:
# loads up Key which combines the document name with the date
key = pd.read_csv(path+'key_1.csv')
# loads up train which has the historical data for websites and number of visits on that day
train = pd.read_csv(path+'train_1.csv')
# submission example has ID and the estimated visits
submission = pd.read_csv(path+'sample_submission_1.csv')

time: 26.7 s


In [44]:
submission

,Id,Visits
0,bf4edcf969af,0
1,929ed2bf52b9,0
2,ff29d0f51d5c,0
3,e98873359be6,0
4,fa012434263a,0
5,48f1e93517a2,0
6,5def418fcb36,0
7,77bd08134351,0
8,5889e6dbb16f,0
9,5f21fef1d764,0


time: 11.8 ms


## Get the Weekdays

In [45]:
## sets up the list of dates
column_dates=list(train[0:0])
## Cuts off the top
column_dates=column_dates[1:]

time: 1.62 ms


In [46]:
# Sets up the dependencies for the todayofweek function
from datetime import date
import calendar
my_date = date.today()
calendar.day_name[my_date.weekday()]

'Saturday'

time: 2.94 ms


In [47]:
# Turns the date into the day of the week
def todayofweek(i):
    year, month, day = (int(x) for x in i.split('-'))    
    answer = datetime.date(year, month, day).weekday()
    #dateValue.DayOfWeek
    #return (calendar.day_name[answer])
    return (answer)
    

time: 4.1 ms


In [48]:
# gets a list of 10 for testing.
for i in column_dates[1:10]:
    print (todayofweek(i))

3
4
5
6
0
1
2
3
4
time: 2.27 ms


## Calculate Daily Averages

In [49]:
# drops off the Page column
traindrop=train.drop('Page', axis=1)

time: 158 ms


In [50]:
#traindrop

time: 1.67 ms


In [51]:
## Creates an array for the weekdays
WeekArray=[0,0,0,0,0,0,0]

time: 1.8 ms


In [52]:
## Gets the number of Weekdays in the dataset for the first row
for i in traindrop[:1]:
    WeekArray[todayofweek(i)]=WeekArray[todayofweek(i)]+1

time: 4.51 ms


In [53]:
# Here are the numbers for them
# First row should be [78, 78, 79, 79, 79, 79, 78]
print(WeekArray)

[78, 78, 79, 79, 79, 79, 78]
time: 1.82 ms


#### Reset and try to add the first set
We don't really want the number of days. We want the average number of visits per day. So lets find that one out. 

In [54]:
#WeekArray is the number of days for each week
#WeekArrayCount is the number of those days
WeekArray=[0,0,0,0,0,0,0]
WeekArrayCount=[0,0,0,0,0,0,0]

time: 1.87 ms


In [55]:
# I get confused, and switch it to a simple list
trains=traindrop[0:1].values[:,:]
trains=list(trains[0])

time: 2.24 ms


In [56]:
# List of values 
# trains

time: 498 µs


In [57]:
#just grabs the first one for testing
trains[0]

18.0

time: 1.86 ms


In [58]:
## Gets the number of Weekdays in the dataset
count = 0
for i in traindrop[:1]:
    WeekArray[todayofweek(i)]=WeekArray[todayofweek(i)]+trains[count]
    WeekArrayCount[todayofweek(i)]=WeekArrayCount[todayofweek(i)]+1
    count=count+1

time: 6.91 ms


In [59]:
print ("The total vists per each day: "+str(WeekArray))
print ("The total of days of the week: "+str(WeekArrayCount))

# Will get the average for each day, rounding up
for i in range (0,7):
    WeekArray[i]=math.ceil(WeekArray[i]/WeekArrayCount[i])
    
print ("Average for each day: "+str(WeekArray))

The total vists per each day: [1376.0, 1931.0, 1707.0, 1576.0, 1732.0, 1810.0, 1834.0]
The total of days of the week: [78, 78, 79, 79, 79, 79, 78]
Average for each day: [18, 25, 22, 20, 22, 23, 24]
time: 4.66 ms


## Averages for all

In [60]:
# probably is the better way to get the required values for all of the pages on dates
# remember traindrop is just the values withouth the page
valuesforall=traindrop.values

time: 1.26 ms


In [61]:
valuesforall

array([[ 18.,  11.,   5., ...,  19.,  18.,  20.],
       [ 11.,  14.,  15., ...,  45.,  26.,  20.],
       [  1.,   0.,   1., ...,   3.,   4.,  17.],
       ..., 
       [ nan,  nan,  nan, ...,  nan,  nan,  nan],
       [ nan,  nan,  nan, ...,  nan,  nan,  nan],
       [ nan,  nan,  nan, ...,  nan,  nan,  nan]])

time: 3 ms


In [62]:
# The nan values are confusing. You cannot do an equal. 
#I found np.isnan to determine if they are nans
print (str((valuesforall[4][4]))+" is nan="+str(np.isnan(valuesforall[4][4])))
print (str((valuesforall[0][4]))+" is nan="+str(np.isnan(valuesforall[0][4])))

nan is nan=True
14.0 is nan=False
time: 2.58 ms


In [63]:
len(train.Page)

145063

time: 6.57 ms


In [64]:
## Creates an array to get an average for each page for the weekdays
## pages is equal to the length - 1 = Top is the header
## 7days is just the 7 days you can 
sevendays, pages = 7, (len(train.Page));
DailyAveragesArray = [[0 for x in range(sevendays)] for y in range(pages)] 

time: 238 ms


#
for i in range(0,(pages-1)): #145061ish remove 1 more 
    if i%10000 == 0:
        print (i/10000)
    WeekArrayCount=[0,0,0,0,0,0,0]
    for s in range(0,len(valuesforall[1])):
        if (np.isnan(valuesforall[i][s])):
            break
        else:
            DailyAveragesArray[i][todayofweek(datetimes[s])]=DailyAveragesArray[i][todayofweek(datetimes[s])]+valuesforall[i][s]
            WeekArrayCount[todayofweek(datetimes[s])]=WeekArrayCount[todayofweek(datetimes[s])]+1
    for s in range(0,7):
        if(WeekArrayCount[s]==0):
            DailyAveragesArray[i][s]=0
        else:
            DailyAveragesArray[i][s]=math.ceil(DailyAveragesArray[i][s]/WeekArrayCount[s])
        

In [65]:
DailyAveragesArray[14]

[0, 0, 0, 0, 0, 0, 0]

time: 2.05 ms


In [66]:
# I want to save this!
#np.save(path+"weights1",DailyAveragesArray)

time: 550 µs


In [68]:
# Loads my array back out
# WEgihts2 is the best
DailyAveragesArray = np.load(path+"weights2.npy")

time: 4.62 ms


In [69]:
#just a check to make sure it happened all the way through
DailyAveragesArray[37206][6]

0

time: 1.71 ms


### Putting it back in
Now that we have the average for each day of the week, we will put that into 

In [70]:
# lets pull out an Id
key.Id[0]

'bf4edcf969af'

time: 28.5 ms


In [71]:
# lets get the number of Pages
len(key.Page)

8703780

time: 1.95 ms


In [72]:
# the simple solution is to run two parellel lists
# this takes a long time because its too simple
# if there are no backups running start from an empty list
backupnamer = list()
backupvisiter = list()

time: 1.15 ms


In [93]:
len(backupvisiter)

5050000

time: 1.55 ms


In [94]:
# starts initialization from the latest backup as available
namer = list(backupnamer)
visiter = list(backupvisiter)

for i in range((len(backupnamer)),8703779): #8703779 or len(key.Page) -1
    # Want to make sure its still running
    if i%50000 == 0:
        print (str(i/10000))
        np.save((path+"visiter"+str(i/10000)),visiter)
        np.save((path+"namer"+str(i/10000)),namer)
    # The page has both the name and the date; Lets split them out
    name=(key.Page[i])[:len(key.Page[i])-11]
    date=(key.Page[i])[len(key.Page[i])-10:]
    
    #print (name+" "+date)
    #print (key.Id[i])
    
    #removing exception to go faster?
    out=DailyAveragesArray[(list(train.Page)).index(name)][todayofweek(date)]
    
    #There is an exception I am fighting through
    #try:
        # We want to get the correct day of the week from the DailyAveragesArray we built 
        # We take two inputs: name = (list(train.Page)).index(name)
        #                     date = todayofweek(date)
        # out will be the average for that day of the week
        #out=DailyAveragesArray[(list(train.Page)).index(name)][todayofweek(date)]
    #except:
        # We ran into an exception at 2264820
        # throwing an exception for the time being and marking it
        #out = -1
        #print ("exception at " + str(len(visiter)))
    
    # Building our submission list 
    #Name side
    namer.append(str(key.Id[i]))
    #Visiter estimation
    visiter.append(str(out))

505.0
510.0
515.0
520.0
525.0
530.0
535.0
540.0
545.0
550.0
555.0
560.0
565.0
570.0
575.0
580.0
585.0
590.0
595.0
600.0
605.0
610.0
615.0
620.0
625.0
630.0
635.0
640.0
645.0
650.0
655.0
660.0
665.0
670.0
675.0
680.0
685.0
690.0
695.0
700.0
705.0
710.0
715.0
720.0
725.0
730.0
735.0
740.0
745.0
750.0
755.0
760.0
765.0
770.0
775.0
780.0
785.0
790.0
795.0
800.0
805.0
810.0
815.0
820.0
825.0
830.0
835.0
840.0
845.0
850.0
855.0
860.0
865.0
870.0
time: 6h 15min 24s


In [95]:
# save a copy at exception
backupvisiter=visiter
backupnamer=namer

time: 1.44 ms


In [96]:
# Make sure both lengths are the same
print (str(len(namer))+" "+str(len(namer)))

8703779 8703779
time: 2.79 ms


In [97]:
# Save the lists
np.save(path+"visiterFinal",visiter)
np.save(path+"namerFinal",namer)

time: 2.3 s


In [92]:
# Loads the lists
backupvisiter = np.load(path+"visiter505.0.npy")
backupnamer = np.load(path+"namer1580505.0.npy")

time: 90.1 ms


In [140]:
len(visiter)

4702740

time: 1.7 ms


## Make the submission

In [135]:
# Makes the submission from the lists
subm = pd.DataFrame({'Id': namer, 'Visits': visiter})

time: 1.26 ms


In [127]:
# Look at the submission quick
subm

,Id,Visits
0,bf4edcf969af,4
1,929ed2bf52b9,2
2,ff29d0f51d5c,3
3,e98873359be6,5
4,fa012434263a,3
5,48f1e93517a2,7
6,5def418fcb36,4
7,77bd08134351,4
8,5889e6dbb16f,2
9,5f21fef1d764,3


time: 6.11 ms


## Goof around with numbers
Not really important anymore

In [56]:
len(submission)

8703780

time: 1.58 ms


In [57]:
# Lists all of the ids
ids = key.Id.values
# lists all of the pages
pages = key.Page.values

d_pages = {}
# d_pages has the id and the page name
for id, page in zip(ids, pages):
    d_pages[id] = page[:-11]

time: 4.01 s


In [58]:
pages = train.Page.values

time: 708 µs


In [59]:
# list of all the wikipedia pages
pages[1]

'2PM_zh.wikipedia.org_all-access_spider'

time: 7.08 ms


In [60]:
visits = np.nan_to_num(np.round(np.nanmedian(train.drop('Page', axis=1).values[:, -56:], axis=1)))

time: 502 ms


/home/jd/.local/lib/python3.5/site-packages/numpy/lib/function_base.py:3858: RuntimeWarning: All-NaN slice encountered
  r = func(a, **kwargs)


In [62]:
visits

array([ 18.,  19.,   4., ...,   0.,   0.,   0.])

time: 1.97 ms


In [61]:
d_visits = {}
for page, visits_number in zip(pages, visits):
    d_visits[page] = visits_number

time: 45.1 ms


In [ ]:
print('Modifying sample submission...')
ss_ids = submission.Id.values
ss_visits = submission.Visits.values


In [ ]:
## Assings a value to each result
for i, ss_id in enumerate(ss_ids):
    ss_visits[i] = (d_visits[d_pages[ss_id]])


In [ ]:
subm = pd.DataFrame({'Id': ss_ids, 'Visits': ss_visits})

In [ ]:
print (subm)

## Submit

In [ ]:
# Makes the submission name out of folders
numFiles=len(os.listdir(path+'submits/'))
subm_name = 'submission'+str(numFiles)+'.csv.gz'
submission=subm

In [ ]:
submission.to_csv((path+'submits/'+subm_name), index=False, compression='gzip')

In [ ]:
#FileLink('/data/webTraffic/submits/'+subm_name)

In [ ]:
print ('http://localhost:8888/view/data/webTraffic/submits/'+subm_name)